In [1]:
import pandas as pd
import random
import numpy as np
import datetime

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
INPUT_FILE = '../toxicity_ratings - twitter.csv'

In [4]:
df = pd.read_csv(INPUT_FILE)
len(df)

/Users/shagunjhaver/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,7,8,9,10,14,15,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


72947

In [5]:
df = df.dropna()
len(df)

70208

In [6]:
toxic_fields = [
    'toxic_score',
    'toxic_score.1',   
    'toxic_score.2',
    'toxic_score.3',    
    'toxic_score.4',    
]

In [7]:
def get_avg_toxicity(row):
    myList = [row[field] for field in toxic_fields]
    result = sum(myList)/20.0
    for field in toxic_fields:
        maxValue = max(myList)
        minValue = min(myList)
        if (maxValue - minValue > 1):
            return None
    return result

In [8]:
df['avg_toxic_score'] = df.apply (lambda row: get_avg_toxicity(row), axis=1)

In [9]:
df = df.dropna()
len(df)

27994

In [10]:
def get_bucket(row, field):
    if (row[field] <= 0.2):
        return 0.0
    elif (row[field] < 0.4):
        return 0.2
    elif (row[field] < 0.6):
        return 0.4
    elif (row[field] < 0.8):
        return 0.6
    else:
        return 0.8

In [11]:
df['avg_toxic_window'] = df.apply(lambda row: get_bucket(row, 'avg_toxic_score'), axis=1)

In [12]:
myrange = [0.2, 0.4, 0.6, 0.8]
sampled_dfs = []
for ow in myrange:
    g = df[df['avg_toxic_window'] == ow]
    print (ow, len(g))
    sampled_dfs.append(g.sample(20))

0.2 599
0.4 356
0.6 141
0.8 143


In [13]:
final_df = pd.concat(sampled_dfs)

In [14]:
def get_random_date():
    random_number_of_days = random.randrange(10)
    random_date = datetime.date(2021, 10, 1) + datetime.timedelta(days=random_number_of_days)
    return random_date

In [15]:
def assignPeripherals(final_df):
    final_df = final_df[['comment', 'avg_toxic_score', 'comment_ID']]
    
    # Set username
    usernameDf = pd.read_csv('usernames.csv')
    usernames = set(list(usernameDf['username']))
    final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
    
    # Set full name
    nameDf = pd.read_csv('random_names.csv')
    names = set(list(nameDf['name']))
    final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)    
    
    # Set integer fields
    final_df['num_likes'] = np.random.randint(1, 100, final_df.shape[0])    
    final_df['num_comments'] = np.random.randint(1, 100, final_df.shape[0])    
    final_df['num_retweets'] = np.random.randint(1, 100, final_df.shape[0])        
    
    # Set publication date
    final_df['pub_date'] = df.apply(lambda row: get_random_date(), axis=1)
    
    return final_df

In [16]:
final_df = assignPeripherals(final_df)
final_df.head()

<ipython-input-15-61dd48437cb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,comment,avg_toxic_score,comment_ID,author,author_name,num_likes,num_comments,num_retweets,pub_date
27991,SHOULD BE A LAW THAT OLD PEOPLE WEAR BRAS after all YOUR HUSBAND WEARS A GIRDLE,0.35,3162_16,ValiantlyThorough,Kirsten Grant,96,58,50,2021-10-07
43907,You don’t feel for the 80+ children? What a pig you are president Don the con.,0.30,2226_18,meaningfully_usable_airtime,Roland Payne,30,51,4,2021-10-08
52658,Oh trust me Trump is killing everything here. That will be next,0.30,5707_17,oblivious_transmitter,Tyler Dixon,46,2,28,2021-10-05
12626,Banter comes with sports. If you're losing sleep over what someone had to say on social media then you have issues. What did you think we weren't gonna talk shit? It's the softies like you that can't take it and go fishing for screenshots.,0.35,916_17,DisgustingPermission,Tara Higgins,93,48,12,2021-10-06
53670,Liberals just can't get any Dumber! Oh Wait !,0.25,4839_20,HushedQuorum,Owen Adams,71,38,64,2021-10-08


In [17]:
final_df = final_df.sample(frac=1).reset_index(drop=True)
final_df.to_csv('main_comments.csv', index=False)

In [18]:
nontoxic_df = df.query('avg_toxic_window == 0.0')

In [19]:
nontoxic_sample = nontoxic_df.sample(30)

In [20]:
nontoxic_sample = assignPeripherals(nontoxic_sample)
nontoxic_sample.head()

<ipython-input-15-61dd48437cb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,comment,avg_toxic_score,comment_ID,author,author_name,num_likes,num_comments,num_retweets,pub_date
10994,😳😳 what if i build you a minecraft house where we can see the sun going down and zombies can't reach us AND we're both girls... ha ha just kidding... unless??,0.00,1199_8,elasticoperation,Victor Harper,34,23,64,2021-10-07
66491,Can't wait for season two!!! 🥰🥰🥰,0.00,4294_22,PM_ME_YOUR_MAYORS,Steven Craig,74,89,85,2021-10-10
51253,Young everybody in here crying 😭😭😭....Awwww🥴,0.05,5206_21,depressedgallantry09,Arianna Evans,40,48,77,2021-10-01
60261,Iran may STRIKE at Imperialist powers by collecting toll in Hormuz Strait @FortRussNews,0.05,5288_22,orderly_reviewer,Tyler Dixon,65,35,75,2021-10-02
14682,"You are my sunshine My only sunshine You make me happy When skies are gray You’ll never know dear, How much I loV̵̤̹͔̐̓̍̾̅e you",0.00,170_1,depressedgallantry09,Adelaide Harrison,57,94,66,2021-10-03


In [21]:
nontoxic_sample.to_csv('nontoxic_comments.csv', index=False)